In [1]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
sess = tf.InteractiveSession()
import os
import numpy as np

In [2]:
# Define functions

def conv2d(x, W, dy ,dx):
  return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def max_pool_axb(x,a,b):
  return tf.nn.max_pool(x, ksize=[1, a, b, 1],
                        strides=[1, a, b, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [3]:
# Define the parameters of the network

#-----------ACC 0.774 *** score 81----------------
nx_conv1=16      # filter_width (frequency)
my_conv1=4   # filter_height (time)
dnx_conv1=4    # stride_width
dmy_conv1=1    # stride_height

nx_pool1=4      # pooling width
my_pool1=1      # pooling height

channels_conv1=32    # channels num

nx_conv2=2
my_conv2=16
dnx_conv2=1
dmy_conv2=8

nx_pool2=2
my_pool2=1


channels_conv2=16

size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
num_neurons_fc1=64

print(size_input_fc1)

1024


In [4]:
# Define the parameters of the network



#-----------ACC 0.774 *** score 81----------------
# nx_conv1=16      # filter_width (frequency)
# my_conv1=4   # filter_height (time)
# dnx_conv1=4    # stride_width
# dmy_conv1=1    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=1      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=1


# channels_conv2=16

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=64

# print(size_input_fc1)

#-----------ACC 0.65 *** score 74----------------
# nx_conv1=16      # filter_width
# my_conv1=4   # filter_height
# dnx_conv1=4    # stride_width
# dmy_conv1=1    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=4      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=2


# channels_conv2=4

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=32

# print(size_input_fc1)

#--------------ACC 0.64  acore 69--------------
# nx_conv1=8      # filter_width
# my_conv1=8  # filter_height
# dnx_conv1=2    # stride_width
# dmy_conv1=2    # stride_height

# nx_pool1=2      # pooling width
# my_pool1=2      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=4
# my_conv2=4
# dnx_conv2=2
# dmy_conv2=2

# nx_pool2=2
# my_pool2=2


# channels_conv2=4

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=32

# print(size_input_fc1)



In [35]:
x = tf.placeholder("float", shape=[None, 128*128])
y_ = tf.placeholder("float", shape=[None, 2])


x_image = tf.reshape(x, [-1,128,128,1])

W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
b_conv1 = bias_variable([channels_conv1])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) + b_conv1)
h_pool1 = max_pool_axb(h_conv1, nx_pool1, my_pool1)


W_conv2 = weight_variable([nx_conv2, my_conv2, channels_conv1, channels_conv2])
b_conv2 = bias_variable([channels_conv2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dnx_conv2, dmy_conv2 ) + b_conv2)
h_pool2 = max_pool_axb(h_conv2, nx_pool2, my_pool2)



W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
b_fc1 = bias_variable([num_neurons_fc1])

h_pool2_flat = tf.reshape(h_pool2, [-1, size_input_fc1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


W_fc2 = weight_variable([num_neurons_fc1, 2])
b_fc2 = bias_variable([2])



keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


In [13]:
testData=sio.loadmat('C:/OSU/SongMat/log/Test/532522 SakiZ - osu!memories.mat')
inputTest=testData['input'].reshape(-1,128*128)
targetTest=testData['target'].reshape(-1,4)
targetTest2=np.zeros([targetTest.shape[0],2])
targetTest2[:,0]=targetTest[:,0]+targetTest[:,1]+targetTest[:,2]
targetTest2[:,1]=targetTest[:,3]
NTest=targetTest2.shape[0]

In [50]:


# merged = tf.summary.merge_all()
# train_writer = tf.summary.FileWriter('/train',sess.graph)
# test_writer = tf.summary.FileWriter('/test')
# tf.global_variables_initializer().run()

sess.run(tf.global_variables_initializer())


In [51]:
saver = tf.train.Saver()


In [52]:
step_size=300
 
dataList=os.listdir('C:/OSU/SongMat/log/Train/')
acc=np.zeros([100,1])

for n in range(int(np.size(dataList)/1)-15):
    inputOsu=np.empty([0,128*128]);
    outputOsu=np.empty([0,4]);
    for k in range(n,n+15):
        
        data=sio.loadmat('C:/OSU/SongMat/log/Train/'+dataList[k])
        inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
        outputOsu=np.append(outputOsu,data['target1'].reshape(-1,4),axis=0)
               
        outputOsu2=np.zeros([outputOsu.shape[0],2]);
        outputOsu2[:,0]=outputOsu[:,0]+outputOsu[:,1]+outputOsu[:,2]
        outputOsu2[:,1]=outputOsu[:,3]
        
    N=inputOsu.shape[0]
    print(N)
    #shuffle the order
    order = np.arange(N-3000)
    np.random.shuffle(order)
    
    for i in range(N-3000):
        if i%step_size == 0:
            
            train_accuracy = accuracy.eval(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 1.0})
#             print("step %d, training accuracy %g" %(i, train_accuracy))
            train_step.run(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: outputOsu2[order[i:i+step_size],:], keep_prob: 0.7})
    
    
    # Test accuracy
    orderAcc = np.arange(NTest)
    np.random.shuffle(orderAcc)
    accTest=accuracy.eval(feed_dict={x: inputTest[orderAcc[0:500],:], y_: targetTest2[orderAcc[0:500],:], keep_prob: 1.0});
    print("test accuracy %g"%accTest)
    print(y_conv.eval(feed_dict={x: inputTest[orderAcc[0:10],:], keep_prob: 1.0}))
    acc[n]=accTest
    print(n)
    
    if acc[n]>0.76:
        break
        
        

21236
test accuracy 0.68
[[ 0.38745388  0.61254615]
 [ 0.69841611  0.30158392]
 [ 0.26792046  0.73207951]
 [ 0.46484497  0.53515506]
 [ 0.41109255  0.58890742]
 [ 0.09869582  0.90130413]
 [ 0.30402526  0.69597477]
 [ 0.71946317  0.2805368 ]
 [ 0.36769575  0.63230419]
 [ 0.47579357  0.5242064 ]]
0
24198
test accuracy 0.68
[[ 0.5726741   0.42732587]
 [ 0.39475173  0.60524827]
 [ 0.77722341  0.22277656]
 [ 0.201416    0.79858398]
 [ 0.82097393  0.17902611]
 [ 0.48280185  0.51719815]
 [ 0.36090603  0.63909394]
 [ 0.20820019  0.79179984]
 [ 0.28175703  0.71824294]
 [ 0.19711991  0.80288011]]
1
23732
test accuracy 0.694
[[ 0.79459405  0.20540595]
 [ 0.55030692  0.44969314]
 [ 0.40110058  0.59889948]
 [ 0.83323777  0.16676223]
 [ 0.67327952  0.32672048]
 [ 0.25865835  0.74134165]
 [ 0.25851393  0.74148601]
 [ 0.47177476  0.52822518]
 [ 0.66896731  0.33103266]
 [ 0.94396043  0.0560396 ]]
2
22078
test accuracy 0.626
[[  9.99998569e-01   1.37229927e-06]
 [  7.30643868e-01   2.69356132e-01]
 [  8

In [53]:
save_path = saver.save(sess, "D:/OSU/results/save/",global_step=n)

In [54]:
print(acc)

[[ 0.68000007]
 [ 0.68000001]
 [ 0.69400001]
 [ 0.62600005]
 [ 0.67000002]
 [ 0.64999998]
 [ 0.71000004]
 [ 0.69599998]
 [ 0.65799999]
 [ 0.69999999]
 [ 0.71200001]
 [ 0.68800002]
 [ 0.73199999]
 [ 0.68400002]
 [ 0.66000003]
 [ 0.74199998]
 [ 0.68200004]
 [ 0.73399997]
 [ 0.73600006]
 [ 0.71599996]
 [ 0.79799998]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     

In [55]:
print(y_conv.eval(feed_dict={x: inputOsu[200:210:1,:], keep_prob: 1.0}))
print ("test accuracy %g"%accuracy.eval(feed_dict={
  x: inputOsu[-500:,:], y_: outputOsu2[-500:,:], keep_prob: 1.0}))

[[ 0.99720156  0.00279847]
 [ 0.01349146  0.98650861]
 [ 0.92983752  0.07016251]
 [ 0.0819061   0.91809386]
 [ 0.96492362  0.03507635]
 [ 0.10546983  0.89453018]
 [ 0.99031061  0.00968945]
 [ 0.1095457   0.89045423]
 [ 0.99059546  0.0094046 ]
 [ 0.05066239  0.9493376 ]]
test accuracy 0.842


In [59]:
# Save the output of new maps 

dataList=os.listdir('C:/OSU/SongMat/log/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if not dataList[i][0]=='y':
        data=sio.loadmat('C:/OSU/SongMat/log/Create/'+dataList[i])
        print(data['osuFolder'])
        
        inputOsuNew=data['input'].reshape(-1,128*128)
        outputOsuNew=data['target']
        outputOsu2New=np.zeros([outputOsuNew.shape[0],2]);
        outputOsu2New[:,0]=outputOsuNew[:,0]+outputOsuNew[:,1]+outputOsuNew[:,2]
        outputOsu2New[:,1]=outputOsuNew[:,3]
        
        y_result=np.empty([0,2])
        
        step_size=500
        cursor=0
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsuNew[-500:,:], y_: outputOsu2New[-500:,:], keep_prob: 1.0}))
        while cursor<inputOsuNew.shape[0]:
            if inputOsuNew[cursor:,:].shape[0]>step_size:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            else:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            cursor+=step_size
            
        data_save={'y2':y_result}
        sio.savemat('C:/OSU/SongMat/log/Create/y2_'+dataList[i],data_save)


[ 'D:\\Program Files (x86)\\osu!\\Songs\\395046 Nakashinoda Mugi - Senjou no Aria\\']
test accuracy 0.646
[ 'D:\\Program Files (x86)\\osu!\\Songs\\395147 TRakker - Countdown to the Blue\\']
test accuracy 0.77
['D:\\Program Files (x86)\\osu!\\Songs\\395149 Asterisk - Rain\\']
test accuracy 0.914
[ 'D:\\Program Files (x86)\\osu!\\Songs\\395252 Hayami Saori - Sono Koe ga Chizu ni Naru (TV EDIT)\\']
test accuracy 0.872
['D:\\Program Files (x86)\\osu!\\Songs\\395298 Nightwish - Sagan\\']
test accuracy 0.754
[ 'D:\\Program Files (x86)\\osu!\\Songs\\395359 mimimemeMIMI - Sayonara Usotsuki\\']
test accuracy 0.468
[ 'D:\\Program Files (x86)\\osu!\\Songs\\395496 Yukinoshita Yukino (CVHayami Saori) & Yuigahama Yui (CVTouyama Nao) - Kimi to\\']
test accuracy 0.656
[ 'D:\\Program Files (x86)\\osu!\\Songs\\395642 Sakamoto Maaya - Okaerinasai (tomatomerde Remix)\\']
test accuracy 0.844
['D:\\Program Files (x86)\\osu!\\Songs\\395657 Rib - Yonjuunana\\']
test accuracy 0.796
[ 'D:\\Program Files (x86)\\

In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
checkpoint_dir='D:/OSU/results/innocence/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)